# Simple AI Agent for SQL queries

In this notebook we will build a simple AI Agent to answer the questions about Baseball players. The knowledge about the baseball players is coming from an internal SQL engine with data about the players' personal information and salaries. 

We will:

* [Create a local SQL Database](#create-local-sql-database)
* [Create SQL tools to help the LLM query out Database](#create-sql-tools)
* [Create the AI Agent Flow](#llm-flow-with-tools-ai-agent)
* [Apply First Tool Use Request from LLM](#first-tool_user-request-from-the-llm-schema)
* [Apply Second Tool Use Request from LLM](#second-tool_user-request-from-the-llm-sql-query)
* [Final reply from the LLM](#final-reply-from-the-llm-based-on-the-tools-used)

### Visual improvements

We will use [rich library](https://github.com/Textualize/rich), and `rich-theme-manager` to make the output more readable, and supress warning messages.

In [1]:
from rich.console import Console
from rich_theme_manager import Theme, ThemeManager
import pathlib

theme_dir = pathlib.Path("themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# Create a console with the dark theme
console = Console(theme=dark)

In [2]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Create local SQL database

We will load a couple of CSV files and create a local SQL database from their data.

In [13]:
import sqlite3
import pandas as pd

def create_database_from_csv(db_name, csv_files):
    """Creates a SQLite database from multiple CSV files."""

    conn = sqlite3.connect(db_name)

    for csv_file in csv_files:
        # Read CSV into a DataFrame
        df = pd.read_csv(csv_file)

        # Get table name from CSV filename (without extension)
        table_name = csv_file.split('/')[-1].split('.')[0]

        # Create table and load data
        df.to_sql(table_name, conn, if_exists='replace', index=False)

    conn.close()



In [14]:

csv_files = ['data/player.csv', 'data/salary.csv']  # Replace with your CSV file names
db_name = 'baseball.db'
create_database_from_csv(db_name, csv_files)

In [16]:
conn = sqlite3.connect(db_name)
conn.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
conn.execute("SELECT count(*) FROM player").fetchall()

[(18846,)]

In [20]:
from dotenv import load_dotenv

load_dotenv()

True

In [58]:
# user_prompt = "In which US state most baseball players were born in 1990?"
user_prompt = "In which US state most baseball players who were in the top 100 richest player in 2010 were born?"

In [59]:
# Now time to connect to the large language model
from openai import OpenAI

openai_client = OpenAI()
completion = openai_client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "system", 
            "content": "You are chatbot, who is helping people with answers to their questions."
        },
        {
            "role": "user", 
            "content": 
                [
                    {"type": "text", "text": user_prompt},
                ],
        },
    ]
)

In [60]:
from rich.panel import Panel
from rich.text import Text

response_text = Text(completion.choices[0].message.content)
styled_panel = Panel(
    response_text,
    title=f"{user_prompt}",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

╭─────── In which US state most baseball players who were in the top 100 richest player in 2010 were born? ───────╮
│                                                                                                                 │
│ I'm sorry, I don't have specific information about where the top 100 richest baseball players in 2010 were      │
│ born. Generally speaking, California, Texas, and Florida are U.S states that have produced many professional    │
│ baseball players due to large populations and year-round playable weather, but I can't confirm these states for │
│ the specific year and criteria you mentioned without access to detailed records or statistics. For the most     │
│ accurate information, I recommend checking dedicated sports statistics websites or databases.                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The LLM can't really help us.

## Create SQL tools

If the naiive approach doesn't work, let's build an AI agent that will help us solve such tasks. First we need a couple of `Tools` that can query the SQL database. There are two main tools that we need:
* Schema Tool - that will describe to the LLM what is the schema of the database
* SQL Query Tool - that will execute the SQL query that is requested by the LLM

In [61]:
tools = [
    {
        "name": "get_db_schema",
        "description": "Describe the schema of the SQLite database, including table names, and column names and types.",
        "input_schema": {
            "type": "object",
            "properties": {}
        }
    },
    {
        "name": "execute_sql_query",
        "description": "Return the query results of the given SQL query to the SQLite database.",
        "input_schema":  {
            "type": "object",
            "properties": {
                "sql_query": {
                    "type": "string",
                    "description": "The sql query to execute against the SQLite database"
                },
            },
            "required": ["sql_query"]
        }
    }
]

In [62]:
user_initial_message = {
    "role": "user", 
    "content": [
        {
            "type": "text",
            "text": user_prompt,
        }
    ]
}

In [63]:
messages = []
messages.append(user_initial_message)

## LLM Flow with Tools (=AI Agent)

We will ask the LLM again, but this time we will provide it with a tool.

In [64]:
import anthropic

anthropic_client = anthropic.Anthropic()

In [65]:
system_prompt = "You are an expert business analyst with deep knowledge of SQL."

In [66]:
response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20241022",
    system=system_prompt,
    max_tokens=8192,
    messages=messages,
    tools=tools
)

In [67]:
console.print(response)

Message(
    id='msg_01VXzGnDumPQcwaUXZaHgFmp',
    content=[
        TextBlock(
            text="Let me help you analyze this. First, I'll check the database schema to understand how to query 
this information.",
            type='text'
        ),
        ToolUseBlock(id='toolu_014Bf99aPM5opmWVzTvjPHXv', input={}, name='get_db_schema', type='tool_use')
    ],
    model='claude-3-5-sonnet-20241022',
    role='assistant',
    stop_reason='tool_use',
    stop_sequence=None,
    type='message',
    usage=Usage(input_tokens=497, output_tokens=62)
)

### First tool_user request from the LLM (schema)

We will append the reply from the LLM to the list of messages that includes the initial user questions. We will need it the list of the messages for the next time we call the LLM.

In [68]:
messages.append({"role": "assistant", "content": response.content})

In [69]:
if response.stop_reason == "tool_use":
    tool_use = next(block for block in response.content if block.type == "tool_use")
    tool_name = tool_use.name
    tool_input = tool_use.input
    tool_use_id = tool_use.id

    print(f"\nTool Used: {tool_name}")
    print("Tool Input:")
    print(tool_input)

    # code_interpreter_results = process_tool_call(e2b_code_interpreter, tool_name, tool_input)



Tool Used: get_db_schema
Tool Input:
{}


We will execute the tool per the LLM request here to understand the flow.

In [70]:
import json

def get_db_schema():
    conn = sqlite3.connect('baseball.db')
    cursor = conn.cursor()
    
    # Get schema for player table
    cursor.execute("PRAGMA table_info(player)")
    player_schema = cursor.fetchall()
    
    # Get schema for salary table 
    cursor.execute("PRAGMA table_info(salary)")
    salary_schema = cursor.fetchall()
    
    conn.close()
    
    return json.dumps({
        'player': player_schema,
        'salary': salary_schema
    })


In [71]:
tool_response = {
    "role": "user",
    "content": [
        {
        "type": "tool_result",
        "tool_use_id": tool_use.id,
        "content": get_db_schema()
        }
    ]
}

### Tool Response

We will append the tool response to the messages list to be sent to the LLM.

In [72]:
messages.append(tool_response)

In [73]:
response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20241022",
    system=system_prompt,
    max_tokens=8192,
    messages=messages,
    tools=tools
)

In [74]:
console.print(response)

Message(
    id='msg_01QLqfyLVDDfAnKzJCDQnT6a',
    content=[
        TextBlock(
            text="Now I'll write a query to find the US state where most of the top 100 highest-paid players in 
2010 were born. We'll need to:\n1. Join the salary and player tables\n2. Filter for the year 2010\n3. Get the top 
100 players by salary\n4. Count players by birth state\n5. Order by the count to find the state with most 
players\n\nHere's the query:",
            type='text'
        ),
        ToolUseBlock(
            id='toolu_01LyspxHTd4Dh4YtpjCUUi8G',
            input={
                'sql_query': "WITH top_100_2010 AS (\n    SELECT p.*, s.salary\n    FROM salary s\n    JOIN player 
p ON s.player_id = p.player_id\n    WHERE s.year = 2010\n    ORDER BY s.salary DESC\n    LIMIT 100\n)\nSELECT 
birth_state, COUNT(*) as player_count\nFROM top_100_2010\nWHERE birth_country = 'USA'\nGROUP BY birth_state\nORDER 
BY player_count DESC\nLIMIT 1;"
            },
            name='execute_sql_query',
            type='tool_use'
        )
    ],
    model='claude-3-5-sonnet-20241022',
    role='assistant',
    stop_reason='tool_use',
    stop_sequence=None,
    type='message',
    usage=Usage(input_tokens=995, output_tokens=272)
)

### Second tool_user request from the LLM (SQL Query)

The LLM analyzed the schema of our database and generated a SQL to answer the user request based on that schema.

In [75]:
messages.append({"role": "assistant", "content": response.content})

In [76]:
if response.stop_reason == "tool_use":
    tool_use = next(block for block in response.content if block.type == "tool_use")
    tool_name = tool_use.name
    tool_input = tool_use.input
    tool_use_id = tool_use.id

    print(f"\nTool Used: {tool_name}")
    print("Tool Input:")
    print(tool_input)

    # code_interpreter_results = process_tool_call(e2b_code_interpreter, tool_name, tool_input)



Tool Used: execute_sql_query
Tool Input:
{'sql_query': "WITH top_100_2010 AS (\n    SELECT p.*, s.salary\n    FROM salary s\n    JOIN player p ON s.player_id = p.player_id\n    WHERE s.year = 2010\n    ORDER BY s.salary DESC\n    LIMIT 100\n)\nSELECT birth_state, COUNT(*) as player_count\nFROM top_100_2010\nWHERE birth_country = 'USA'\nGROUP BY birth_state\nORDER BY player_count DESC\nLIMIT 1;"}


In [77]:
def execute_sql_query(sql_query):
    import json
    import sqlite3
    
    # Connect to SQLite database
    conn = sqlite3.connect('baseball.db')
    cursor = conn.cursor()
    
    # Execute query
    cursor.execute(sql_query)
    
    # Fetch results
    results = cursor.fetchall()
    
    # Get column names
    column_names = [description[0] for description in cursor.description]
    
    # Convert results to list of dicts
    results_list = []
    for row in results:
        row_dict = dict(zip(column_names, row))
        results_list.append(row_dict)
        
    # Close connection
    conn.close()
    
    # Return JSON string
    return json.dumps(results_list)


Let's check the SQL query that was generated by the LLM:

In [88]:
from rich.syntax import Syntax

syntax = Syntax(tool_input['sql_query'], "sql", line_numbers=True)
console.print(syntax)

   1 WITH top_100_2010 AS (                                                                                        
   2     SELECT p.*, s.salary                                                                                      
   3     FROM salary s                                                                                             
   4     JOIN player p ON s.player_id = p.player_id                                                                
   5     WHERE s.year = 2010                                                                                       
   6     ORDER BY s.salary DESC                                                                                    
   7     LIMIT 100                                                                                                 
   8 )                                                                                                             
   9 SELECT birth_state, COUNT(*) as player_count                                                                  
  10 FROM top_100_2010                                                                                             
  11 WHERE birth_country = 'USA'                                                                                   
  12 GROUP BY birth_state                                                                                          
  13 ORDER BY player_count DESC                                                                                    
  14 LIMIT 1;                                                                                                      

In [79]:
tool_response = {
    "role": "user",
    "content": [
        {
        "type": "tool_result",
        "tool_use_id": tool_use.id,
        "content": execute_sql_query(tool_input['sql_query'])
        }
    ]
}

### Final reply from the LLM based on the tools used

In [80]:
messages.append(tool_response)

In [81]:
response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20241022",
    system=system_prompt,
    max_tokens=8192,
    messages=messages,
    tools=tools
)

In [82]:
console.print(response)

Message(
    id='msg_01MpFVCeNhBUw94P8rvaM4fd',
    content=[
        TextBlock(
            text='Based on the query results, California (CA) was the birth state of most baseball players who were
among the top 100 highest-paid players in 2010. Specifically, 15 players from the top 100 highest-paid players in 
2010 were born in California.',
            type='text'
        )
    ],
    model='claude-3-5-sonnet-20241022',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=Usage(input_tokens=1292, output_tokens=66)
)

And we got the response to our question from the LLM based on query to an internal database, without the need to know any SQL.

In [89]:
response_text = Text(response.content[0].text)
styled_panel = Panel(
    response_text,
    title=f"{user_prompt}",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

╭─────── In which US state most baseball players who were in the top 100 richest player in 2010 were born? ───────╮
│                                                                                                                 │
│ Based on the query results, California (CA) was the birth state of most baseball players who were among the top │
│ 100 highest-paid players in 2010. Specifically, 15 players from the top 100 highest-paid players in 2010 were   │
│ born in California.                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯